In [0]:
!pip install pymysql

In [0]:
import pymysql.cursors

import urllib
import time
 
from urllib.request import urlopen
from bs4 import BeautifulSoup

import pandas as pd

import time

주가 Crawling

In [0]:
stockItem = '005930'
stockName = 'Samsung_Electronics'
url = 'http://finance.naver.com/item/sise_day.nhn?code='+ stockItem
html = urlopen(url) 
source = BeautifulSoup(html.read(), "html.parser")
 
maxPage=source.find_all("table",align="center")
mp = maxPage[0].find_all("td",class_="pgRR")
mpNum = 10#int(mp[0].a.get('href')[-3:])    ## 크롤링할 페이지수 한페이지에 10 영업일씩
lst = []

for page in range(1, mpNum+1):
    #print (str(page) )
    url = 'http://finance.naver.com/item/sise_day.nhn?code=' + stockItem +'&page='+ str(page)
    html = urlopen(url)
    source = BeautifulSoup(html.read(), "html.parser")
    srlists=source.find_all("tr")
    isCheckNone = None

    if((page % 1) == 0):
        time.sleep(1.50)

    for i in range(1,len(srlists)-1):
        if(srlists[i].span != isCheckNone):

            #srlists[i].td.text
            lst.append(("-".join(srlists[i].find_all("td",align="center")[0].text.split(".")), int(srlists[i].find_all("td",class_="num")[0].text.replace(',','')) ))
df = pd.DataFrame(lst)
df.columns = ['date','price']

STOCK 이라는 이름의 DATABASE 생성

In [0]:
conn = pymysql.connect(host='34.84.209.183',
        user='root',
        password='mysql',
        charset='utf8mb4')

try:
    with conn.cursor() as cursor:
        sql = 'DROP DATABASE STOCK'
        cursor.execute(sql)
        #result = cursor.fetchall()
        #print(result)
    conn.commit()
except:
    print('db exist')
finally:
    conn.close()

In [0]:
conn = pymysql.connect(host='34.84.209.183',
        user='root',
        password='mysql',
        charset='utf8mb4')

try:
    with conn.cursor() as cursor:
        sql = 'CREATE DATABASE STOCK'
        cursor.execute(sql)
        #result = cursor.fetchall()
        #print(result)
except:
    print('db exist')
finally:
    conn.close()

In [0]:
conn = pymysql.connect(host='34.84.209.183',user='root',password='mysql',charset='utf8mb4',db = 'STOCK')

try:
    with conn.cursor() as cursor:
        sql = '''CREATE TABLE STOCK_PRICE (
                COMPANY_ID int(11) NOT NULL ,
                COMAPNY_NAME varchar(255) NOT NULL,
                DATE DATETIME NOT NULL ,
                END_PRICE INT(11) NOT NULL,
                PRIMARY KEY(COMPANY_ID,DATE)
               )'''
        cursor.execute(sql)
        #result = cursor.fetchall()
        #print(result)
except:
    print('table exist')
finally:
    conn.close()

테이블에 값 채워넣기

In [0]:
sql = "INSERT INTO STOCK_PRICE \n VALUES"
for i in df.values:
    
    sql = sql + "(%d, '%s', STR_TO_DATE('%s','%%Y-%%m-%%d'), %d),\n"%(int(stockItem),stockName,i[0],i[1])

conn = pymysql.connect(host='34.84.209.183',user='root',password='mysql',charset='utf8mb4',db = 'STOCK')

try:
    with conn.cursor() as cursor:
        cursor.execute(sql[:-2])
    conn.commit()
finally:
    conn.close()

In [0]:
conn = pymysql.connect(host='34.84.209.183',user='root',password='mysql',charset='utf8mb4',db = 'STOCK')

try:
    with conn.cursor() as cursor:
        sql = 'select * from STOCK_PRICE'
        cursor.execute(sql)
        result = cursor.fetchall()
        #print(result)
    
finally:
    conn.close()

((5930, 'Samsung_Electronics', datetime.datetime(2019, 4, 29, 0, 0), 46150), (5930, 'Samsung_Electronics', datetime.datetime(2019, 4, 30, 0, 0), 45850), (5930, 'Samsung_Electronics', datetime.datetime(2019, 5, 2, 0, 0), 45900), (5930, 'Samsung_Electronics', datetime.datetime(2019, 5, 3, 0, 0), 45300), (5930, 'Samsung_Electronics', datetime.datetime(2019, 5, 7, 0, 0), 44850), (5930, 'Samsung_Electronics', datetime.datetime(2019, 5, 8, 0, 0), 44250), (5930, 'Samsung_Electronics', datetime.datetime(2019, 5, 9, 0, 0), 42450), (5930, 'Samsung_Electronics', datetime.datetime(2019, 5, 10, 0, 0), 42900), (5930, 'Samsung_Electronics', datetime.datetime(2019, 5, 13, 0, 0), 42650), (5930, 'Samsung_Electronics', datetime.datetime(2019, 5, 14, 0, 0), 42650), (5930, 'Samsung_Electronics', datetime.datetime(2019, 5, 15, 0, 0), 42550), (5930, 'Samsung_Electronics', datetime.datetime(2019, 5, 16, 0, 0), 41550), (5930, 'Samsung_Electronics', datetime.datetime(2019, 5, 17, 0, 0), 41200), (5930, 'Samsung_

In [0]:
pd.DataFrame(list(result))

,0,1,2,3
0,5930,Samsung_Electronics,2019-04-29,46150
1,5930,Samsung_Electronics,2019-04-30,45850
2,5930,Samsung_Electronics,2019-05-02,45900
3,5930,Samsung_Electronics,2019-05-03,45300
4,5930,Samsung_Electronics,2019-05-07,44850
5,5930,Samsung_Electronics,2019-05-08,44250
6,5930,Samsung_Electronics,2019-05-09,42450
7,5930,Samsung_Electronics,2019-05-10,42900
8,5930,Samsung_Electronics,2019-05-13,42650
9,5930,Samsung_Electronics,2019-05-14,42650
